In [ ]:
import os
import pandas as pd
import torch
from model_tit import *
from timeit import default_timer as timer
import util
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SequentialSampler


os.environ['CUDA_VISIBLE_DEVICES'] = '0'


# start here ! ###################################################################################
def do_predict(net, tokenizer, valid_loader):

    text = []

    start_timer = timer()
    valid_num = 0
    for t, batch in enumerate(valid_loader):
        batch_size = len(batch['image'])
        image = batch['image'].cuda()


        net.eval()
        with torch.no_grad():
            k = net.forward_argmax_decode(image)
            k = k.data.cpu().numpy()
            k = tokenizer.predict_to_inchi(k)
            text.extend(k)

        valid_num += batch_size
        print('\r %8d / %d  %s' % (valid_num, len(valid_loader.dataset), util.time_to_str(timer() - start_timer, 'sec')),
              end='', flush=True)

    assert(valid_num == len(valid_loader.dataset))
    print('')
    return text


#fold = 0
out_dir = '/content/drive/MyDrive/molecule/server_code/model/tit0'.format(fold)
initial_checkpoint = '/Users/zhenyuanzhang/Desktop/udemy/AWS_Lambda_and_Serverless/IMG2InChi/100000_model.pth'#None #

is_norm_ichi = False #True
if 1:

    ## setup  ----------------------------------------
    mode = 'local'

    submit_dir = '/Users/zhenyuanzhang/Desktop/udemy/AWS_Lambda_and_Serverless/IMG2InChi'
    os.makedirs(submit_dir, exist_ok=True)

    ## dataset ------------------------------------
    tokenizer = util.load_tokenizer()
    df_valid = pd.read_pickle('/content/drive/MyDrive/molecule/server_code/data/df_test.pkl')

    valid_dataset = util.BmsDataset(df_valid, tokenizer, mode = 'test', augment=util.rot_augment)
    valid_loader  = DataLoader(
        valid_dataset,
        sampler = SequentialSampler(valid_dataset),
        #sampler = util.FixNumSampler(valid_dataset, 100),
        batch_size  = 64, #32,
        drop_last   = False,
        num_workers = 8,
        pin_memory  = True,
        collate_fn  = lambda batch: util.collate_fn(batch,False)
    )

    start_timer = timer()

    tokenizer = util.load_tokenizer()
    net = Net().cuda()
    net.load_state_dict(torch.load(initial_checkpoint)['state_dict'], strict=True)

    #---
    predict = do_predict(net, tokenizer, valid_loader)

    #print('time taken : %s\n\n' % util.time_to_str(timer() - start_timer, 'min'))
    #----


    # if is_norm_ichi:
    #     predict = [normalize_inchi(t) for t in predict]  #

    # df_submit = pd.DataFrame()
    # df_submit.loc[:,'image_id'] = df_valid.image_id.values
    # df_submit.loc[:,'InChI'] = predict #
    # df_submit.to_csv(submit_dir + '/186000submit.csv', index=False)

    # print('submit_dir : %s\n' % (submit_dir))
    # print('initial_checkpoint : %s\n' % (initial_checkpoint))
    # print('df_submit : %s\n' % str(df_submit.shape))
    # print('%s\n' % str(df_submit))
    # print('\n')

    print(predict)

if __name__ == '__main__':
    pass
